# Multiwavelength cutout

In [173]:
import importlib as imp
import astroquery
imp.reload(astroquery)
import astroquery.herschel.higal
imp.reload(astroquery.herschel.higal)
import pylab as pl
pl.rcParams['figure.dpi'] = 150

In [174]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [175]:
%run /orange/adamginsburg/mgps/mgps/analysis/make_sed_cutout_image.py

In [176]:
paths.basepath = '/orange/adamginsburg/mgps/mgps/'

In [177]:
coord = SkyCoord("17:45:57.7530532310 -28:57:10.7694483833", unit=(u.h, u.deg), frame='icrs')
coord.galactic

<SkyCoord (Galactic): (l, b) in deg
    (0.0246741, -0.0727428)>

### Final Version

In [178]:
ims = {
    #'4-9 keV': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/acisi-band3-4.fits',
    #'3.6um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I1.fits',
    #'4.5um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I2.fits',
    #'5.8um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I3.fits',
    #'8.0um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I4.fits',
    # '3.6um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I1_13368832_0000_6_E8709676_maic.fits',
    # '4.5um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I2_13368832_0000_6_E8709929_maic.fits',
    # '5.8um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I3_13368832_0000_6_E8709933_maic.fits',
    # '8.0um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I4_13368832_0000_6_E8709940_maic.fits',
    
    '3.6um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I1_13368832_0000_6_E8709676_maic.fits',
    '4.5um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I2_13368832_0000_6_E8709929_maic.fits',
    '5.8um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I3_13368832_0000_6_E8709933_maic.fits',
    '8.0um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I4_13368832_0000_6_E8709940_maic.fits',
    '24um': '/orange/adamginsburg/cmz/mipsgal_24micron_data/gc_mosaic_MIPSGAL_gal.fits',
    '70um': '/orange/adamginsburg/cmz/herschel/destripe_l000_blue_wgls_rcal.fits',
    '160um': '/orange/adamginsburg/cmz/herschel/destripe_l000_red_wgls_rcal.fits',
    '250um': '/orange/adamginsburg/cmz/herschel/destripe_l000_PSW_wgls_rcal.fits',
    #'450um': '/orange/adamginsburg/cmz/scuba2/450um_map.fits',
    #'350um': '/orange/adamginsburg/cmz/herschel/destripe_l000_PMW_wgls_rcal.fits',
    #'500um': '/orange/adamginsburg/cmz/herschel/destripe_l000_PLW_wgls_rcal.fits',
    '850um': '/orange/adamginsburg/ACES/ancillary/data/2017.1.01185.S/science_goal.uid___A001_X12a3_Xe7/group.uid___A001_X12a3_Xe8/member.uid___A001_X12a3_Xe9/calibrated/50_kms_cloud_sci.spw0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15.mfs.I.manual.image.pbcor.fits',
    '3mm': '/orange/adamginsburg/ACES/mosaics/continuum/12m_continuum_commonbeam_circular_reimaged_mosaic_MUSTANGfeathered.fits',
    '6cm': '/orange/adamginsburg/cmz/xinglu/SgrA_CONT_tclean_nterm2.image.tt0.fits',
    '20cm': '/orange/adamginsburg/cmz/meerkat/MeerKAT_Galactic_Centre_1284MHz-StokesI.fits'
}

In [179]:
import matplotlib.colors as mcolors
import numpy as np
from astropy.visualization import simple_norm

colors1 = pl.cm.gray_r(np.linspace(0., 1, 128))
colors2 = pl.cm.hot(np.linspace(0, 1, 128))

colors = np.vstack((colors1, colors2))
mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

In [180]:
import regions

In [181]:
region = regions.RectangleSkyRegion(center=coord, width=15*u.arcsec, height=15*u.arcsec)
cutregion = regions.CircleSkyRegion(center=coord, radius=15*u.arcsec) # circumscribe the above region so we can match RA/Dec to Galactic

In [182]:
bigregion = regions.RectangleSkyRegion(center=coord, width=35*u.arcsec, height=35*u.arcsec)
bigcircregion = regions.CircleSkyRegion(center=coord, radius=35*u.arcsec)

In [183]:
biggerregion = regions.RectangleSkyRegion(center=coord, width=65*u.arcsec, height=65*u.arcsec)

In [184]:
from astropy.wcs import WCS

In [185]:
def galactic_cutout(hdu, reg):
    import reproject.mosaicking, reproject
    csys, sz = reproject.mosaicking.find_optimal_celestial_wcs(hdu, frame='galactic')
    newdata, _ = reproject.reproject_interp(input_data=hdu,
                                            output_projection=csys,
                                            shape_out=sz)
    preg = reg.to_pixel(csys)
    msk = preg.to_mask()
    slcs,_ = msk.get_overlap_slices(newdata.shape)
    new_hdu = fits.PrimaryHDU(data=newdata[slcs], header=csys[slcs].to_header())
    return new_hdu, csys[slcs]

### Multipanel Image

In [ ]:
pl.figure(figsize=(15,12), dpi=150)
ii = 1
for wl, fn in ims.items():
    fh = fits.open(fn)
    ww = WCS(fh[0].header).celestial
    
    msk = bigcircregion.to_pixel(ww).to_mask()
    slcs, _ = msk.get_overlap_slices(fh[0].data.squeeze().shape)
    co = msk.cutout(fh[0].data.squeeze())

    hdu, ww = galactic_cutout((co, ww[slcs]), bigregion)
    
    ax = pl.subplot(3,4,ii, projection=ww)
    ax.imshow(hdu.data, cmap=mymap)
    ax.scatter_coord(coord, marker='o', facecolor='none', edgecolor='g', s=100)
    #pl.xticks([])
    #pl.yticks([])
    #pl.xlabel("")
    #pl.ylabel("")
    lon = ax.coords[0]
    lat = ax.coords[1]
    if ii == 9:
        lon.set_axislabel('Galactic Longitude')
        lat.set_axislabel('Galactic Latitude')
    else:
        lon.set_ticks_visible(False)
        lon.set_ticklabel_visible(False)
        lat.set_ticks_visible(False)
        lat.set_ticklabel_visible(False)
        lon.set_axislabel('')
        lat.set_axislabel('')
    ax.set_title(wl.replace('u',r'$\mu$'))
    ii += 1
    print(ii, end=", ")

pl.tight_layout()
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/continuum_cutouts.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/continuum_cutouts.png', bbox_inches='tight')

2, 3, 4, 5, 

### Herschel upper limits

In [ ]:
from radio_beam import Beam
import radio_beam

In [ ]:
import latex_info

In [ ]:
from astropy.table import Table
centrfittbl = Table.read('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/centroid_fits.ecsv')
centrfittbl.add_index('Image Data')
centrfittbl.add_index('Image Type')
b7r = centrfittbl.loc['Continuum'].loc['Image Type', 'B7']
b3r = centrfittbl.loc['Continuum'].loc['Image Type', 'spw33+35']
spectral_index = (
    np.log(b7r['Amp[Jy]'] / b3r['Amp[Jy]']) /
    np.log((350*u.GHz/(102*u.GHz)).decompose().value))
spectral_index

In [ ]:
from astropy.table import Table
centrfittbl = Table.read('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/centroid_fits.ecsv')
centrfittbl.add_index(['Image Data', 'Image Type'])

In [ ]:
import glob
from spectral_cube import SpectralCube

In [ ]:
path = '/orange/adamginsburg/ACES/data/2021.1.00172.L/science_goal.uid___A001_X1590_X30a8/group.uid___A001_X1590_X30a9/member.uid___A001_X15a0_X13c/calibrated/working/'
imgs = sorted(glob.glob(f"{path}/*.cube.*.image"))
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubes = [SpectralCube.read(fn) for fn in imgs[:1]]
try:
    b3beam = cubes[0].beams.common_beam()
except AttributeError:
    b3beam = cubes[0].beam

In [ ]:
fns_2017 = glob.glob("/orange/adamginsburg/ACES/ancillary/data/2017.1.01185.S/science_goal.uid___A001_X12a3_Xe7/group.uid___A001_X12a3_Xe8/member.uid___A001_X12a3_Xe9/calibrated/*cube*image.pbcor")
scubes2017 = [SpectralCube.read(fn, format='casa_image').subcube_from_regions([region]) for fn in fns_2017[:1]]
b7beam = scubes2017[0].beam

In [ ]:
centrfittbl

In [ ]:
from radio_beam import Beam
import radio_beam
from astropy.wcs import WCS

In [ ]:
herschelspitzer = {
    #'3.6um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I1.fits',
    #'4.5um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I2.fits',
    #'5.8um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I3.fits',
    #'8.0um': '/orange/adamginsburg/cmz/glimpse_data/GLM_00000+0000_mosaic_I4.fits',
    # '3.6um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I1_13368832_0000_6_E8709676_maic.fits',
    # '4.5um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I2_13368832_0000_6_E8709929_maic.fits',
    # '5.8um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I3_13368832_0000_6_E8709933_maic.fits',
    # '8.0um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/cutout_SPITZER_I4_13368832_0000_6_E8709940_maic.fits',
    '3.6um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I1_13368832_0000_6_E8709676_maic.fits',
    '4.5um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I2_13368832_0000_6_E8709929_maic.fits',
    '5.8um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I3_13368832_0000_6_E8709933_maic.fits',
    '8.0um': '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SPITZER_stolovy_I4_13368832_0000_6_E8709940_maic.fits',
    '24um': '/orange/adamginsburg/cmz/mipsgal_24micron_data/gc_mosaic_MIPSGAL_gal.fits',
    '70um': '/orange/adamginsburg/cmz/herschel/destripe_l000_blue_wgls_rcal.fits',
    '160um': '/orange/adamginsburg/cmz/herschel/destripe_l000_red_wgls_rcal.fits',
    '250um': '/orange/adamginsburg/cmz/herschel/destripe_l000_PSW_wgls_rcal.fits',
    '350um': '/orange/adamginsburg/cmz/herschel/destripe_l000_PMW_wgls_rcal.fits',
    '500um': '/orange/adamginsburg/cmz/herschel/destripe_l000_PLW_wgls_rcal.fits',
    #'450um': '/orange/adamginsburg/cmz/scuba2/450um_map.fits',
    #'850um': '/orange/adamginsburg/cmz/scuba2/850um_map.fits',
    '60000um': '/orange/adamginsburg/cmz/xinglu/SgrA_CONT_tclean_nterm2.image.tt0.fits',
    '200000um': '/orange/adamginsburg/cmz/meerkat/MeerKAT_Galactic_Centre_1284MHz-StokesI.fits'
}
# beams from Traficante+ 2011 (Except Spitzer)
herschelspitzer_resolution = {
    '3.6um': Beam(2*u.arcsec),
    '4.5um': Beam(2*u.arcsec),
    '5.8um': Beam(2*u.arcsec),
    '8.0um': Beam(2*u.arcsec),
    '24um': Beam(6*u.arcsec),
    '70um': Beam(10.7*u.arcsec, 9.7*u.arcsec),
    '160um': Beam(13.9*u.arcsec, 13.2*u.arcsec),
    '250um': Beam(23.9*u.arcsec, 22.8*u.arcsec),
    '350um': Beam(31.3*u.arcsec, 29.3*u.arcsec),
    '500um': Beam(43.8*u.arcsec, 41.1*u.arcsec),
    '450um': Beam(6.5*u.arcsec),
    '850um': Beam(12*u.arcsec),
    '60000um': radio_beam.Beam.from_fits_header(fits.getheader('/orange/adamginsburg/cmz/xinglu/SgrA_CONT_tclean_nterm2.image.tt0.fits')),
    '200000um': radio_beam.Beam.from_fits_header(fits.getheader('/orange/adamginsburg/cmz/meerkat/MeerKAT_Galactic_Centre_1284MHz-StokesI.fits'))
}

latexdict_ulim = latex_info.latexdict.copy()
latexdict_ulim['header_start'] = r'\label{tab:sed}'
latexdict_ulim['caption'] = 'Spectral Energy Distribution'
latexdict_ulim['preamble'] = r'\centering'
latexdict_ulim['tablefoot'] = '\par\n' + r'Except for the ALMA measurements at 3 mm and 850 \um, these values are upper limits.'
formats = {'Wavelength': '0.1f',
           'Surface Brightness': '0.3f',
           'Beam Area': '0.3e',
           'Flux': '0.3g'
          }

ulimtbl = []

for wl, fn in herschelspitzer.items():
    fh = fits.open(fn)
    ww = WCS(fh[0].header).celestial
    xx, yy = np.array(list(map(int, ww.world_to_pixel(coord))))
    #print(crd)
    val = fh[0].data[yy, xx]
    if u.Quantity(wl) < 1*u.mm:
        print(wl, val, (u.Quantity(val, u.MJy/u.sr)*herschelspitzer_resolution[wl].sr).to(u.Jy))
        ulimtbl.append((u.Quantity(wl.strip('um'), u.um), val*u.MJy/u.sr,
                        herschelspitzer_resolution[wl].sr,
                        (u.Quantity(val, u.MJy/u.sr)*herschelspitzer_resolution[wl].sr).to(u.Jy)))
    else:
        beam = herschelspitzer_resolution[wl]
        print(wl, ((u.Quantity(val, u.Jy))/beam.sr).to(u.MJy/u.sr), (u.Quantity(val, u.Jy)), )
              
        ulimtbl.append([u.Quantity(wl.strip('um'), u.um),
                        val*u.Jy/beam.sr,
                        beam.sr,
                        u.Quantity(val, u.Jy)])

b7ind = (centrfittbl['Image Data'] == 'Continuum') & (centrfittbl['Image Type'] == 'B7')
b3ind = (centrfittbl['Image Data'] == 'Continuum') & (centrfittbl['Image Type'] == 'spw33+35')

assert centrfittbl[b7ind]['Amp[Jy]'][0]*u.Jy > 90*u.mJy

ulimtbl.append([850*u.um,
                centrfittbl[b7ind]['Amp[Jy]'][0]*u.Jy/b7beam.sr,
                b7beam.sr,
                centrfittbl[b7ind]['Amp[Jy]'][0]*u.Jy])
ulimtbl.append([(102*u.GHz).to(u.um, u.spectral()), 
                centrfittbl[b3ind]['Amp[Jy]'][0]*u.Jy/b3beam.sr,
                b3beam.sr,
                centrfittbl[b3ind]['Amp[Jy]'][0]*u.Jy])

# SMA data 4/19/2024
b6bm = radio_beam.Beam(3*u.arcsec) # GUESS
ulimtbl.append([(230*u.GHz).to(u.um, u.spectral()), 
                16.6*u.mJy/b6bm.sr,
                b6bm,
                16.6*u.mJy])
    
ulimtbl = Table(rows=ulimtbl, names=['Wavelength', 'Surface Brightness', 
                                     'Beam Area',
                                     'Flux'])
ulimtbl.sort('Wavelength')
ulimtbl.write('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED.tex',
              latexdict=latexdict_ulim,
              formats=formats,
              overwrite=True)
ulimtbl

In [ ]:
from astropy.modeling.models import BlackBody
import dust_emissivity

In [ ]:
ulwl = ulimtbl['Wavelength']
pl.plot(ulwl[ulwl < 800*u.um], ulimtbl['Flux'][ulwl < 800*u.um], 'v', markerfacecolor='none', markeredgecolor='k')
pl.plot(ulwl[ulwl > 1*u.cm], ulimtbl['Flux'][ulwl > 1*u.cm], 'v', markerfacecolor='none', markeredgecolor='k')
pl.plot((102*u.GHz).to(u.um, u.spectral()), centrfittbl['Amp[Jy]'][b3ind], 's', markeredgecolor='k', markerfacecolor='b')
pl.plot(850*u.um, centrfittbl['Amp[Jy]'][b7ind], 's', markeredgecolor='k', markerfacecolor='b')
nurange = np.geomspace(10*u.GHz, 100*u.THz, 1000)
beta = 1.5
for tem, ls in zip((15, 25, 50, 75), ('-', '--', '-.', ':')):
    bm = b3beam.sr
    column = dust_emissivity.dust.colofsnu(nu=102*u.GHz,
                                           snu_per_beam=u.Quantity(b3r['Amp[Jy]'], u.Jy)/bm,
                                           beta=beta,
                                           nu0=271.1*u.GHz, 
                                           kappa0=0.0114*u.cm**2*u.g**-1,
                                           temperature=tem*u.K)
    mbb = dust_emissivity.blackbody.modified_blackbody(nurange,
                                                       temperature=tem*u.K,
                                                       beta=beta,
                                                       nu0=271.1*u.GHz, 
                                                       kappa0=0.0114*u.cm**2*u.g**-1*100,
                                                       column=column,
                                                      ).to(u.Jy/u.sr)*bm
    pl.plot(nurange.to(u.um, u.spectral()), mbb.to(u.Jy), linestyle=ls, zorder=-5, label=f'T={tem}\nN={column.value:0.1e}')
pl.loglog();
pl.axis([1,1e5,5e-4,300]);
pl.legend(loc='best',);
pl.xlabel("Wavelength [$\mu$m]")
pl.ylabel("Flux Density $S_\\nu$ [Jy]")
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits.png', bbox_inches='tight')
pl.axis([1,3e5,1e-5,300]);
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA.png', bbox_inches='tight')
pl.plot((230*u.GHz).to(u.um, u.spectral()), (26.0*u.mJy).to(u.Jy), 's', markeredgecolor='k', markerfacecolor='r')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA_SMA.png', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA_SMA.pdf', bbox_inches='tight')

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

In [ ]:
ax = pl.subplot()
# inset axes....
x1, x2, y1, y2 = 400, 500, 2.5e-1, 1.5  # subregion of the original image


ulwl = ulimtbl['Wavelength']
ax.plot(ulwl[ulwl < 800*u.um], ulimtbl['Flux'][ulwl < 800*u.um], 'v', markerfacecolor='none', markeredgecolor='k')
ax.plot(ulwl[ulwl > 1*u.cm], ulimtbl['Flux'][ulwl > 1*u.cm], 'v', markerfacecolor='none', markeredgecolor='k')
ax.plot((102*u.GHz).to(u.um, u.spectral()), centrfittbl['Amp[Jy]'][b3ind], 's', markeredgecolor='k', markerfacecolor='b')
ax.plot(850*u.um, centrfittbl['Amp[Jy]'][b7ind], 's', markeredgecolor='k', markerfacecolor='b')

axins = ax.inset_axes(
    [0.35, 0.1, 0.20, 0.25],
    xlim=(x1, x2), ylim=(y1, y2),
    #xticklabels=[], yticklabels=[]
)

nurange = np.geomspace(10*u.GHz, 100*u.THz, 1000)
beta = 1.5
for tem, ls in zip((15, 25, 50, 75), ('-', '--', '-.', ':')):
    bm = b3beam.sr
    column = dust_emissivity.dust.colofsnu(nu=102*u.GHz,
                                           snu_per_beam=u.Quantity(b3r['Amp[Jy]'], u.Jy)/bm,
                                           beta=beta,
                                           nu0=271.1*u.GHz, 
                                           kappa0=0.0114*u.cm**2*u.g**-1,
                                           temperature=tem*u.K)
    mbb = dust_emissivity.blackbody.modified_blackbody(nurange,
                                                       temperature=tem*u.K,
                                                       beta=beta,
                                                       nu0=271.1*u.GHz, 
                                                       kappa0=0.0114*u.cm**2*u.g**-1*100,
                                                       column=column,
                                                      ).to(u.Jy/u.sr)*bm
    L, = ax.plot(nurange.to(u.um, u.spectral()), mbb.to(u.Jy), linestyle=ls, zorder=-5, label=f'T={tem}\nN={column.value:0.1e}')
    axins.plot(nurange.to(u.um, u.spectral()), mbb.to(u.Jy), linestyle=ls, zorder=-5, 
               color=L.get_color(),
               label=f'T={tem}\nN={column.value:0.1e}')


#ax.indicate_inset_zoom(axins, edgecolor="black", )
blah = mark_inset(ax, axins, loc1=1, loc2=2)

ax.loglog();
ax.axis([1,1e5,5e-4,300]);
pl.legend(loc='best',);
ax.set_xlabel("Wavelength [$\mu$m]")
ax.set_ylabel("Flux Density $S_\\nu$ [Jy]")
# pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits.pdf', bbox_inches='tight')
# pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits.png', bbox_inches='tight')
ax.axis([1,3e5,1e-5,300]);
# pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA.pdf', bbox_inches='tight')
# pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA.png', bbox_inches='tight')
ax.plot((230*u.GHz).to(u.um, u.spectral()), (26.0*u.mJy).to(u.Jy), 's', markeredgecolor='k', markerfacecolor='r')
# pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA_SMA.pdf', bbox_inches='tight')
# pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_upperlimits_VLA_SMA.png', bbox_inches='tight')

#axins.loglog()
axins.semilogx()

# logscale: hide ticks
# holy fucking hackery.  This took absolutely forever to come up with.
axins.yaxis.set_minor_formatter(lambda x, y: "")
axins.xaxis.set_minor_formatter(lambda x, y: "")
axins.yaxis.set_major_formatter(lambda x, y: "")
axins.xaxis.set_major_formatter(lambda x, y: "")

pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_planned_B9.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SED_with_planned_B9.png', bbox_inches='tight')

In [ ]:
axins.yaxis.get_major_formatter()

### Chandra 

In [ ]:
pl.figure(figsize=(12,8), dpi=150)
ii = 1
for fn in glob.glob('/orange/adamginsburg/cmz/chandra/sgra/acis*fits'):
    fh = fits.open(fn)
    ww = WCS(fh[0].header).celestial
    msk = bigregion.to_pixel(ww).to_mask()
    slcs, _ = msk.get_overlap_slices(fh[0].data.squeeze().shape)
    co = msk.cutout(fh[0].data.squeeze())
    ax = pl.subplot(2,3,ii, projection=ww[slcs])
    ax.imshow(co, cmap=mymap)
    ax.scatter_coord(coord, marker='o', facecolor='none', edgecolor='g', s=100)

    lon = ax.coords[0]
    lat = ax.coords[1]
    lon.set_ticks_visible(False)
    lon.set_ticklabel_visible(False)
    lat.set_ticks_visible(False)
    lat.set_ticklabel_visible(False)
    lon.set_axislabel('')
    lat.set_axislabel('')
    ii += 1

pl.tight_layout()
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/chandra_overlays.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/chandra_overlays.png', bbox_inches='tight')

In [ ]:
%%bash
ls /orange/adamginsburg/ACES/data/2021.1.00172.L/science_goal.uid___A001_X1590_X30a8/group.uid___A001_X1590_X30a9/member.uid___A001_X15a0_X13c/calibrated/working/*spw33_35*.image.tt0.pbcor.fits

In [ ]:
fn = "/orange/adamginsburg/ACES/data/2021.1.00172.L/science_goal.uid___A001_X1590_X30a8/group.uid___A001_X1590_X30a9/member.uid___A001_X15a0_X13c/calibrated/working/uid___A001_X15a0_X13c.s36_0.Sgr_A_star_sci.spw33_35.cont.I.iter1.image.tt0.pbcor.fits"
if not os.path.exists(fn):
    fn = "/orange/adamginsburg/ACES/data/2021.1.00172.L/science_goal.uid___A001_X1590_X30a8/group.uid___A001_X1590_X30a9/member.uid___A001_X15a0_X13c/calibrated/working/uid___A001_X15a0_X13c.s36_0.Sgr_A_star_sci.oldhigh_spw33_35.cont.I.iter1.image.tt0.pbcor.fits"
if not os.path.exists(fn):
    fn = "/orange/adamginsburg/ACES/data/2021.1.00172.L/science_goal.uid___A001_X1590_X30a8/group.uid___A001_X1590_X30a9/member.uid___A001_X15a0_X13c/calibrated/working/uid___A001_X15a0_X13c.s36_0.Sgr_A_star_sci.v1high_spw33_35.cont.I.iter1.image.tt0.pbcor.fits"
contcube = SpectralCube.read(fn)
fh = fits.open(fn)
contwcs = WCS(fh[0].header).celestial
preg = region.to_pixel(contwcs)
msk = preg.to_mask()
contim = msk.cutout(fh[0].data.squeeze());
beam = radio_beam.Beam.from_fits_header(fh[0].header)
jtok = beam.jtok(102*u.GHz).value
contim = contim * jtok
slcs,_ = msk.get_overlap_slices(fh[0].data.squeeze().shape)
contwcs_co = contwcs[slcs] 

In [ ]:
pl.figure(figsize=(12,8), dpi=150)
fn = '/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/acisi-band3-4.fits'
fh = fits.open(fn)
ww = WCS(fh[0].header).celestial
msk = bigregion.to_pixel(ww).to_mask()
slcs, _ = msk.get_overlap_slices(fh[0].data.squeeze().shape)
co = msk.cutout(fh[0].data.squeeze())
ax = pl.subplot(1,1,1, projection=ww[slcs])
ax.imshow(co, cmap=mymap)
ax.scatter_coord(coord, marker='o', facecolor='none', edgecolor='g', s=100)
ax.contour(contim, transform=ax.get_transform(contwcs_co),
           levels=[0.04,  0.08, ]);

fhmk = fits.open(ims['20cm'])
ww2 = WCS(fhmk[0].header).celestial
msk = biggerregion.to_pixel(ww2).to_mask()
slcs, _ = msk.get_overlap_slices(fhmk[0].data.squeeze().shape)
co = msk.cutout(fhmk[0].data.squeeze())


lon = ax.coords[0]
lat = ax.coords[1]
#lon.set_ticks_visible(False)
#lon.set_ticklabel_visible(False)
#lat.set_ticks_visible(False)
#lat.set_ticklabel_visible(False)
lon.set_axislabel('Galactic Longitude')
lat.set_axislabel('Galactic Latitude')

pl.tight_layout()
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/chandra_dwang_overlay.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/chandra_dwang_overlay.png', bbox_inches='tight')
ax.contour(co, transform=ax.get_transform(ww2[slcs]), levels=np.linspace(0.005, 0.0068, 6), cmap=pl.cm.Blues)
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/chandra_dwang_overlay_withMEERKAT.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/chandra_dwang_overlay_withMEERKAT.png', bbox_inches='tight')

# Context Images

## CO Context Image

courtesy sofue

In [ ]:
cloudreg = regions.RectangleSkyRegion(coord.galactic, 15*u.arcmin, 15*u.arcmin)
cocube = SpectralCube.read('/orange/adamginsburg/cmz/nobeyama/12CO-2.BEARS.FITS', format='fits').subcube_from_regions([cloudreg])
cocube

In [ ]:
cobearsm0 = cocube.spectral_slab(45*u.km/u.s, 55*u.km/u.s).moment0()
ax = pl.subplot(projection=cobearsm0.wcs.celestial)
im_ = ax.imshow(cobearsm0.value, cmap=mymap)
ax.contour(contim, transform=ax.get_transform(contwcs_co),
           levels=[0.04,  0.08, ],
           colors=['cyan', 'cyan'],
          );
xx, yy = cobearsm0.wcs.celestial.world_to_pixel(coord)
#ax.arrow(xx, yy-20, 0, 15, color='b', width=1)
cb = pl.colorbar(mappable=im_)
cb.set_label("Integrated Intensity [K km s$^{-1}$]")
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/12COContext.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/12COContext.png', bbox_inches='tight')

In [ ]:
cobearsm0 = cocube.moment0()
ax = pl.subplot(projection=cobearsm0.wcs.celestial)
im_ = ax.imshow(cobearsm0.value, cmap=mymap)
ax.contour(contim, transform=ax.get_transform(contwcs_co),
           levels=[0.04,  0.08, ],
           colors=['cyan', 'cyan'],
          );
xx, yy = cobearsm0.wcs.celestial.world_to_pixel(coord)
#ax.arrow(xx, yy-20, 0, 15, color='b', width=1)
cb = pl.colorbar(mappable=im_)
cb.set_label("Integrated Intensity [K km s$^{-1}$]")
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/12COContext_AllVelo.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/12COContext_AllVelo.png', bbox_inches='tight')

## TP

In [ ]:
cube = SpectralCube.read('/orange/adamginsburg/ACES/mosaics/cubes/ACES_TP_spw25_mosaic.fits')
cube.allow_huge_operations = True
cs_tp = cube.with_spectral_unit(u.km/u.s, rest_value=97.98095*u.GHz, velocity_convention='radio').spectral_slab(-300*u.km/u.s, 300*u.km/u.s)
so_tp = cube.with_spectral_unit(u.km/u.s, rest_value=99.29987*u.GHz, velocity_convention='radio').spectral_slab(-300*u.km/u.s, 300*u.km/u.s)
m0cs = cs_tp.to(u.K).moment0(axis=0)
m0so = so_tp.to(u.K).moment0(axis=0)

In [ ]:
cs_tpm0 = cs_tp.subcube_from_regions([cloudreg]).moment0()
ax = pl.subplot(projection=cs_tpm0.wcs.celestial)
im_ = ax.imshow(cs_tpm0.value, cmap=mymap)
ax.contour(contim, transform=ax.get_transform(contwcs_co),
           levels=[0.04,  0.08, ],
           colors=['cyan', 'cyan'],
          );
xx, yy = cs_tpm0.wcs.celestial.world_to_pixel(coord)
#ax.arrow(xx, yy-20, 0, 15, color='b', width=1)
cb = pl.colorbar(mappable=im_)
cb.set_label("Integrated Intensity [K km s$^{-1}$]")
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/CS21TPContext_AllVelo.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/CS21TPContext_AllVelo.png', bbox_inches='tight')

In [ ]:
cs_tpm0 = cs_tp.subcube_from_regions([cloudreg]).spectral_slab(45*u.km/u.s, 55*u.km/u.s).moment0()
ax = pl.subplot(projection=cs_tpm0.wcs.celestial)
im_ = ax.imshow(cs_tpm0.value, cmap=mymap)
ax.contour(contim, transform=ax.get_transform(contwcs_co),
           levels=[0.04,  0.08, ],
           colors=['cyan', 'cyan'],
          );
xx, yy = cs_tpm0.wcs.celestial.world_to_pixel(coord)
#ax.arrow(xx, yy-20, 0, 15, color='b', width=1)
cb = pl.colorbar(mappable=im_)
cb.set_label("Integrated Intensity [K km s$^{-1}$]")
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/CS21TPContext_45to55kms.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/CS21TPContext_45to55kms.png', bbox_inches='tight')

In [ ]:
so_tpm0 = so_tp.subcube_from_regions([cloudreg]).spectral_slab(45*u.km/u.s, 55*u.km/u.s).moment0()
ax = pl.subplot(projection=so_tpm0.wcs.celestial)
im_ = ax.imshow(so_tpm0.value, cmap=mymap)
ax.contour(contim, transform=ax.get_transform(contwcs_co),
           levels=[0.04,  0.08, ],
           colors=['cyan', 'cyan'],
          );
xx, yy = so_tpm0.wcs.celestial.world_to_pixel(coord)
#ax.arrow(xx, yy-20, 0, 15, color='b', width=1)
cb = pl.colorbar(mappable=im_)
cb.set_label("Integrated Intensity [K km s$^{-1}$]")
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SO21TPContext_45to55kms.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/SO21TPContext_45to55kms.png', bbox_inches='tight')

## Mustang

In [ ]:
fhmustang = fits.open('/orange/adamginsburg/ACES/mosaics/continuum/12m_continuum_commonbeam_circular_reimaged_mosaic_MUSTANGfeathered.fits')
wwmustang = WCS(fhmustang[0].header)
cloudmask = cloudreg.to_pixel(wwmustang).to_mask()
slcs,_ = cloudmask.get_overlap_slices(fhmustang[0].data.squeeze().shape)
mustangcld = cloudmask.cutout(fhmustang[0].data)
ax = pl.subplot(projection=wwmustang[slcs])
im_ = ax.imshow(mustangcld, cmap=mymap, norm=simple_norm(mustangcld, max_percent=99.95, min_percent=1, stretch='asinh'))
ax.contour(contim, transform=ax.get_transform(contwcs_co),
           levels=[0.04,  0.08, ],
           colors=['cyan', 'cyan'],
          );
cb = pl.colorbar(mappable = im_)
cb.set_label("S$_{3 \mathrm{mm}}$ [Jy beam$^{-1}$]")
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")

slcs_inset,_ = bigregion.to_pixel(wwmustang[slcs]).to_mask().get_overlap_slices(mustangcld.shape)
brmask = bigregion.to_pixel(wwmustang).to_mask()
slcs,_ = brmask.get_overlap_slices(fhmustang[0].data.squeeze().shape)
axins = ax.inset_axes([0.65, 0.05, 0.3, 0.3],
                      xlim=(slcs_inset[0].start, slcs_inset[0].stop),
                      ylim=(slcs_inset[1].start, slcs_inset[1].stop),
                      #projection=wwmustang[slcs_inset]
                     )
im_ = axins.imshow(mustangcld, cmap=mymap,
                   norm=simple_norm(mustangcld, max_cut=0.13, min_cut=0.036, stretch='asinh'))
axins.set_xticklabels("")
axins.set_yticklabels("")
axins.set_xticks([])
axins.set_yticks([])
# axins.coords[0].set_axislabel("")
# axins.coords[1].set_axislabel("")
# axins.coords[0].set_ticklabel(visible=False)
# axins.coords[1].set_ticklabel(visible=False);
# axins.coords[0].set_ticks_visible(False)
# axins.coords[1].set_ticks_visible(False);
ax.indicate_inset_zoom(axins, edgecolor="black");
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/MUSTANG3mmContext.pdf', bbox_inches='tight')
pl.savefig('/orange/adamginsburg/ACES/broadline_sources/G0.025-0.073/MUSTANG3mmContext.png', bbox_inches='tight')

In [ ]:
pl.imshow(mustangcld[slcs_inset],
          cmap=mymap,
          norm=simple_norm(mustangcld, max_cut=0.13, min_cut=0.036, stretch='asinh'))
pl.colorbar()